In [175]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
from sklearn.impute import SimpleImputer

In [176]:
train=pd.read_csv(r'C:\Users\admin\Desktop\과제 모음집\flex\july-dacon\july-dacon\235745_parking_data\train.csv')
test=pd.read_csv(r'C:\Users\admin\Desktop\과제 모음집\flex\july-dacon\july-dacon\235745_parking_data\test.csv')
submission=pd.read_csv(r'C:\Users\admin\Desktop\과제 모음집\flex\july-dacon\july-dacon\235745_parking_data\sample_submission.csv')

In [177]:
train.shape, test.shape

((2952, 15), (1022, 14))

In [178]:
train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0


In [179]:
test.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0


In [180]:
train.isna().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              0
임대보증금                           569
임대료                             569
도보 10분거리 내 지하철역 수(환승노선 수 반영)    211
도보 10분거리 내 버스정류장 수                4
단지내주차면수                           0
등록차량수                             0
dtype: int64

In [181]:
test.isna().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              2
임대보증금                           180
임대료                             180
도보 10분거리 내 지하철역 수(환승노선 수 반영)     42
도보 10분거리 내 버스정류장 수                0
단지내주차면수                           0
dtype: int64

In [182]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
       '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수'],
      dtype='object')

In [183]:
train.columns=['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '지하철',
       '버스', '단지내주차면수', '등록차량수']

test.columns=['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '지하철', '버스', '단지내주차면수']

In [184]:
local_map={} #지역, 자격유형, 공급유형, 임대건물구분 전부 숫자로 
for i, loc in enumerate(train['지역'].unique()):
    local_map[loc]=i
train['지역']=train['지역'].map(local_map)
test['지역']=test['지역'].map(local_map)

local_type={}
for i, loc in enumerate(train['자격유형'].unique()):
    local_type[loc]=i
train['자격유형']=train['자격유형'].map(local_type)
test['자격유형']=test['자격유형'].map(local_type)

local_type2={}
for i, loc in enumerate(train['공급유형'].unique()):
    local_type2[loc]=i
train['공급유형']=train['공급유형'].map(local_type2)
test['공급유형']=test['공급유형'].map(local_type2)

local_building={}
for i, loc in enumerate(train['임대건물구분'].unique()):
    local_building[loc]=i
train['임대건물구분']=train['임대건물구분'].map(local_building)
test['임대건물구분']=test['임대건물구분'].map(local_building)

In [185]:
train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
0,C2483,900,0,0,0,39.72,134,38.0,0,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,0,0,0,39.72,15,38.0,0,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,0,0,0,51.93,385,38.0,0,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,0,0,0,51.93,15,38.0,0,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,0,0,0,51.93,41,38.0,0,27304000,184330,0.0,3.0,1425.0,1015.0


In [186]:
train['전용면적']=train['전용면적']//5*5
test['전용면적']=test['전용면적']//5*5

In [187]:
train['전용면적']

0        35.0
1        35.0
2        50.0
3        50.0
4        50.0
5        50.0
6        50.0
7        55.0
8        30.0
9        35.0
10       35.0
11       45.0
12       45.0
13       50.0
14       50.0
15       30.0
16       30.0
17       30.0
18       40.0
19       40.0
20       40.0
21       40.0
22       45.0
23       45.0
24       50.0
25       50.0
26       35.0
27       35.0
28       50.0
29       50.0
30       55.0
31       55.0
32       35.0
33       35.0
34       45.0
35       50.0
36       35.0
37       35.0
38       35.0
39       45.0
40       45.0
41       50.0
42       35.0
43       35.0
44       35.0
45       35.0
46       45.0
47       45.0
48       45.0
49       45.0
50       45.0
51       45.0
52       50.0
53       50.0
54       50.0
55       30.0
56       35.0
57       35.0
58       35.0
59       45.0
60       45.0
61       45.0
62       45.0
63       45.0
64       45.0
65       50.0
66       50.0
67       35.0
68       35.0
69       45.0
70       45.0
71    

In [188]:
idx = train[train['전용면적']>100].index #100초과인 것
train.loc[idx, '전용면적'] = 100 #100으로 맞추기
idx = test[test['전용면적']>100].index
test.loc[idx, '전용면적'] = 100

idx = train[train['전용면적']<15].index
train.loc[idx, '전용면적'] = 15
idx = test[test['전용면적']<15].index
test.loc[idx, '전용면적'] = 15

In [189]:
test['전용면적'].unique()

array([ 35.,  45.,  50.,  30.,  55.,  25.,  75., 100.,  15.,  20.,  40.,
        60.,  80.,  70.])

In [190]:
columns = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '지하철',
       '버스', '단지내주차면수']
target = '등록차량수'
area_columns = []
for area in train['전용면적'].unique():
    area_columns.append(f'면적_{area}')

In [191]:
new_train = pd.DataFrame()
new_test = pd.DataFrame()

In [192]:
for i, code in tqdm(enumerate(train['단지코드'].unique())):
    temp = train[train['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
    
    new_train.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']
    
for i, code in tqdm(enumerate(test['단지코드'].unique())):
    temp = test[test['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_test.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()

423it [00:04, 85.25it/s]
150it [00:01, 87.75it/s]


In [193]:
new_train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,면적_35.0,면적_50.0,면적_55.0,면적_30.0,면적_45.0,면적_40.0,면적_25.0,면적_70.0,면적_15.0,면적_20.0,면적_100.0,면적_60.0,면적_75.0,면적_80.0,면적_65.0,등록차량수
0,C2483,900.0,0.0,0.0,0.0,35.0,134.0,38.0,0.0,15667000,103680,0.0,3.0,1425.0,149.0,665.0,86.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1015.0
1,C2515,545.0,0.0,1.0,0.0,30.0,276.0,17.0,0.0,9216000,82940,0.0,3.0,624.0,80.0,132.0,0.0,276.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,205.0
2,C1407,1216.0,0.0,2.0,0.0,30.0,288.0,13.0,0.0,15620000,127350,1.0,1.0,1285.0,0.0,124.0,0.0,390.0,340.0,362.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1064.0
3,C1945,755.0,0.0,3.0,0.0,35.0,120.0,6.0,1.0,15607000,130070,1.0,3.0,734.0,240.0,303.0,212.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,730.0
4,C1470,696.0,0.0,4.0,0.0,35.0,228.0,14.0,0.0,12015000,102130,0.0,2.0,645.0,254.0,246.0,0.0,0.0,196.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,553.0
5,C1898,566.0,0.0,4.0,0.0,35.0,230.0,9.0,0.0,12015000,102130,0.0,6.0,517.0,271.0,86.0,0.0,0.0,209.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,415.0
6,C1244,1722.0,0.0,3.0,0.0,35.0,275.0,10.0,0.0,29551000,203030,0.0,10.0,1483.0,788.0,390.0,0.0,0.0,544.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1804.0
7,C1171,624.0,0.0,2.0,0.0,30.0,190.0,13.0,0.0,17730000,118190,0.0,5.0,634.0,164.0,40.0,0.0,190.0,230.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,700.0
8,C2073,361.0,0.0,5.0,0.0,35.0,94.0,13.0,0.0,10813000,144180,0.0,5.0,288.0,168.0,0.0,40.0,0.0,153.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,301.0
9,C2513,754.0,0.0,6.0,0.0,50.0,517.0,15.0,0.0,32443000,192250,0.0,6.0,530.0,0.0,754.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,820.0


In [195]:
new_train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '지하철', '버스', '단지내주차면수', '면적_35.0', '면적_50.0',
       '면적_55.0', '면적_30.0', '면적_45.0', '면적_40.0', '면적_25.0', '면적_70.0',
       '면적_15.0', '면적_20.0', '면적_100.0', '면적_60.0', '면적_75.0', '면적_80.0',
       '면적_65.0', '등록차량수'],
      dtype='object')

In [196]:
new_train['임대보증금']=new_train['임대보증금'].replace('-',np.nan)
new_train['임대료']=new_train['임대료'].replace('-',np.nan)
new_test['임대보증금']=new_test['임대보증금'].replace('-',np.nan)
new_test['임대료']=new_test['임대료'].replace('-',np.nan)

In [197]:
imputer=SimpleImputer(strategy='most_frequent')
new_train=pd.DataFrame(imputer.fit_transform(new_train))
new_test=pd.DataFrame(imputer.fit_transform(new_test))

In [198]:
x_train = new_train.iloc[:, 1:-1]
y_train = new_train.iloc[:,-1]
x_test = new_test.iloc[:,1:]

In [199]:
model = RandomForestRegressor(n_jobs=-1, random_state=42)

In [200]:
model.fit(x_train, y_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [201]:
pred = model.predict(x_test)

In [202]:
submission['num'] = pred

In [203]:
submission.to_csv('제출용2.csv', index=False)